Pretrain

In [1]:
!git clone https://github.com/JoannaBy/RussianNovels.git

Cloning into 'RussianNovels'...
remote: Enumerating objects: 119, done.
remote: Total 119 (delta 0), reused 0 (delta 0), pack-reused 119 (from 1)
Receiving objects: 100% (119/119), 21.67 MiB | 13.76 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [2]:
import os
import glob

repo_path = 'RussianNovels'
text_files_pattern = os.path.join(repo_path, '**', '*.txt')

# Search for all .txt files in all subdirectories
all_files = glob.glob(text_files_pattern, recursive=True)

# Merge the content of all files into one large string
full_corpus = ""
for file_path in all_files:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            full_corpus += f.read()
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")

# Print corpus size information and save it
print(f"Total files found: {len(all_files)}")
print(f"Total corpus size (in characters): {len(full_corpus)}")

# Save the merged corpus to a single file
output_filename = 'russian_novels_corpus.txt'
with open(output_filename, 'w', encoding='utf-8') as f:
    f.write(full_corpus)

print(f"Corpus saved to file: {output_filename}")

Total files found: 108
Total corpus size (in characters): 45348575
Corpus saved to file: russian_novels_corpus.txt


In [3]:
import re
import os

with open('russian_novels_corpus.txt', 'r', encoding='utf-8') as f:
     full_corpus = f.read()

# Split text into lines (assuming each line/paragraph is a potential unit)
lines = full_corpus.split('\n')
lines = [line.strip() for line in lines if line.strip()]  # Remove empty lines

# Deduplication: Removes completely identical lines/paragraphs
def remove_duplicates(data_list):
    seen = set()
    unique_data = []
    for item in data_list:
        if item not in seen:
            seen.add(item)
            unique_data.append(item)
    return unique_data

lines_unique = remove_duplicates(lines)
print(f"Lines after deduplication: {len(lines_unique)} (Removed: {len(lines) - len(lines_unique)})")

# Non-Cyrillic filtering: Removes lines containing Latin characters
LATIN_PATTERN = re.compile(r'[a-zA-Z]')

def filter_non_cyrillic(data_list):
    filtered_data = [line for line in data_list if not LATIN_PATTERN.search(line)]
    return filtered_data

lines_cyrillic = filter_non_cyrillic(lines_unique)
print(f"Lines after non-Cyrillic filtering: {len(lines_cyrillic)} (Removed: {len(lines_unique) - len(lines_cyrillic)})")

# Handling repeating punctuation and normalization
def normalize_punctuation(text):
    text = re.sub(r'\.{2,}', '...', text) # replace 2 or more dots with '...'
    text = re.sub(r'([?!])\1+', r'\1', text) # '!!!' -> '!', '???' -> '?'
    text = re.sub(r'\s+', ' ', text).strip() # replace multiple spaces with a single space
    text = re.sub(r'\s([,.!?:;])', r'\1', text) # remove extra spaces before punctuation

    return text

lines_normalized = [normalize_punctuation(line) for line in lines_cyrillic]

# Merging cleaned and normalized lines back into a single corpus
cleaned_corpus = '\n'.join(lines_normalized)

print(f"Final corpus size (characters): {len(cleaned_corpus)}")
print("Example of cleaned text (first 500 chars):")
print(cleaned_corpus[:500])

# Saving cleaned corpus for the next step
output_filename_cleaned = 'russian_novels_corpus_cleaned.txt'
with open(output_filename_cleaned, 'w', encoding='utf-8') as f:
    f.write(cleaned_corpus)

print(f"\nCleaned corpus saved to file: {output_filename_cleaned}")

Lines after deduplication: 304839 (Removed: 10652)
Lines after non-Cyrillic filtering: 295808 (Removed: 9031)
Final corpus size (characters): 39657833
Example of cleaned text (first 500 chars):
В начале 1928 года в Берлине знатоку живописи Бруно Кречмару, человеку, очень, кажется, сведущему, но отнюдь не блестящему, пришлось быть экспертом в пустячном, прямо даже глупом деле. Модный художник Кок написал портрет фильмовой артистки Дорианны Карениной. Фирма личных кремов приобретала у нее право помещать на плакатах репродукцию с портрета в виде рекламы своей губной помады. На портрете Дорианна держала прижатой к голому своему плечу большую плюшевую Чипи. Горн из Нью-Йорка тотчас предъяви

Cleaned corpus saved to file: russian_novels_corpus_cleaned.txt


In [4]:
# Tokenizer Setup
from tokenizers import Tokenizer, models, pre_tokenizers, trainers, processors, decoders

# Define the tokenizer architecture (BPE)
tokenizer = Tokenizer(models.BPE())

# Configure the pre-tokenizer to split input using ByteLevel encoding
# This handles raw bytes, allowing the model to process any unicode string
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

SPECIAL_TOKENS = ["<bos>", "<eos>", "<pad>", "<unk>", "<mask>"]

# Define vocabulary size and special tokens
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=SPECIAL_TOKENS)

# Train the tokenizer on the cleaned corpus
tokenizer.train(["russian_novels_corpus_cleaned.txt"], trainer=trainer)

# Post-Processor setup: automatically adds <bos> and <eos> tokens to sequences
# Logic: <bos> + Sequence + <eos>
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"<bos> $A <eos>",
    pair=f"<bos> $A <eos> <eos> $B <eos>",
    special_tokens=[
        ("<bos>", tokenizer.token_to_id("<bos>")),
        ("<eos>", tokenizer.token_to_id("<eos>")),
    ]
)

# Configure the decoder to match the ByteLevel pre-tokenizer
tokenizer.decoder = decoders.ByteLevel()

# Testing the Tokenizer
test_text = "Давайте проверим этот токенайзер."
encoding = tokenizer.encode(test_text)

print(f"Original text: {test_text}")
print(f"Tokens (encoding.tokens): {encoding.tokens}")

# Verify decoding
decoded_text = tokenizer.decode(encoding.ids)
print(f"Decoded text: {decoded_text}")

tokenizer.save("bpe_russian_novels_tokenizer.json")

Original text: Давайте проверим этот токенайзер.
Tokens (encoding.tokens): ['<bos>', 'ÐĶ', 'Ð°Ð²', 'Ð°Ð¹ÑĤÐµ', 'ĠÐ¿ÑĢÐ¾Ð²ÐµÑĢ', 'Ð¸Ð¼', 'ĠÑįÑĤÐ¾ÑĤ', 'ĠÑĤÐ¾Ðº', 'ÐµÐ½', 'Ð°Ð¹', 'Ð·', 'ÐµÑĢ', '.', '<eos>']
Decoded text: Давайте проверим этот токенайзер.


In [5]:
import torch
import os
from datasets import load_dataset, Dataset
from transformers import PreTrainedTokenizerFast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CONTEXT_LENGTH = 512  # Context window size for the model
TOKENIZER_FILE = "bpe_russian_novels_tokenizer.json"
DATA_FILE = "russian_novels_corpus_cleaned.txt"

# Load the pre-trained tokenizer
tokenizer = PreTrainedTokenizerFast(tokenizer_file=TOKENIZER_FILE)

# Define special tokens established during the training phase
tokenizer.bos_token = "<bos>"
tokenizer.eos_token = "<eos>"
tokenizer.pad_token = "<pad>"
tokenizer.unk_token = "<unk>"
tokenizer.mask_token = "<mask>"

# Set pad_token_id explicitly to prevent training errors
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.token_to_id("<pad>")

# Load data into a Hugging Face Dataset object
# 'load_dataset' creates a dataset with a single column named 'text'
raw_datasets = load_dataset('text', data_files={'train': DATA_FILE})

def tokenize_function(examples):
    """Tokenizes the text without truncation."""
    # truncation=False ensures we keep full sequences to group them later into blocks
    return tokenizer(examples["text"], truncation=False)

# Apply tokenization to the entire dataset
tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,                # Process in batches for speed
    num_proc=os.cpu_count(),     # Use all available CPU cores
    remove_columns=["text"],     # Remove the raw text column to save memory
)

# Chunking logic: Groups texts into blocks of size 512
def group_texts(examples):
    """
    Concatenates all texts and splits them into chunks of CONTEXT_LENGTH.
    This prepares the data for Language Modeling (CLM/MLM).
    """
    concatenated_examples = {}

    # Concatenate all 'input_ids' into a single list
    for key, list_of_lists in examples.items():
        concatenated_examples[key] = sum(list_of_lists, [])

    total_length = len(concatenated_examples[list(examples.keys())[0]])

    # Truncate the total length to be divisible by CONTEXT_LENGTH
    total_length = (total_length // CONTEXT_LENGTH) * CONTEXT_LENGTH

    result = {}

    # Split into chunks
    for k, t in concatenated_examples.items():
        chunks_list = []
        for i in range(0, total_length, CONTEXT_LENGTH): # Step size: 512 tokens
            chunk = t[i : i + CONTEXT_LENGTH]
            chunks_list.append(chunk)
        result[k] = chunks_list

    # Add 'labels' column: for Language Modeling, labels are usually identical to input_ids
    result["labels"] = result["input_ids"].copy()
    return result

# Apply the chunking transformation
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=os.cpu_count(),
)

final_train_dataset = lm_datasets["train"]

print(f"Final number of training chunks: {len(final_train_dataset)}")
print(f"Length of a single chunk: {len(final_train_dataset[0]['input_ids'])} tokens")
print("-" * 60)
print("Example of the first chunk (input_ids):")
print(final_train_dataset[0]["input_ids"])
print("Decoded example of the first chunk:")
print(tokenizer.decode(final_train_dataset[0]["input_ids"]))

Generating train split: 0 examples [00:00, ? examples/s]

Map (num_proc=2):   0%|          | 0/295808 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/295808 [00:00<?, ? examples/s]

Final number of training chunks: 19996
Length of a single chunk: 512 tokens
------------------------------------------------------------
Example of the first chunk (input_ids):
[0, 507, 6632, 14420, 27, 2023, 140, 22909, 6495, 5424, 9021, 124, 22697, 165, 5494, 130, 15, 3734, 15, 582, 15, 1472, 15, 6120, 848, 443, 15, 342, 15019, 177, 3838, 443, 15, 4446, 571, 10980, 992, 164, 140, 1498, 2716, 555, 15, 1598, 611, 1518, 164, 1955, 17, 303, 176, 322, 13594, 23294, 5596, 6090, 16093, 146, 1429, 10922, 275, 7578, 124, 208, 196, 2375, 154, 1270, 17, 599, 388, 867, 2565, 415, 14914, 151, 9499, 267, 175, 836, 2767, 3189, 226, 185, 2615, 217, 387, 8031, 491, 730, 3484, 136, 21640, 140, 3387, 19653, 135, 917, 1078, 332, 578, 5244, 17, 946, 4274, 558, 7578, 12573, 9458, 2584, 5635, 149, 392, 363, 2659, 8190, 5264, 448, 1488, 22657, 470, 1131, 124, 17, 11304, 292, 17806, 16, 12287, 183, 276, 1294, 19562, 166, 20715, 1254, 1017, 17, 1, 0, 2090, 478, 1449, 404, 638, 2486, 4376, 1, 0, 14718, 361, 14

In [6]:
from transformers import LlamaForCausalLM, LlamaConfig

configuration = LlamaConfig(hidden_size=1024, intermediate_size=1536, num_hidden_layers=16, num_attention_heads=16, num_key_value_heads=8, vocab_size=25000)
model = LlamaForCausalLM(configuration)
model.to(device)
configuration = model.config

total_params = model.num_parameters()
print(f"Number of model's parameters: {total_params / 1_000_000:.2f}M")

Number of model's parameters: 177.06M


In [7]:
from transformers import TrainingArguments, Trainer, PreTrainedTokenizerFast

CONTEXT_LENGTH = 512
VOCAB_SIZE = 25000

split_datasets = lm_datasets['train'].train_test_split(test_size=0.1)
train_dataset = split_datasets['train']
eval_dataset = split_datasets['test']

BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 16

training_args = TrainingArguments(

    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    weight_decay=0.01,

    num_train_epochs=3,
    learning_rate=5e-5,

    eval_strategy="steps", # Estimation on a validation set
    per_device_eval_batch_size=BATCH_SIZE,

    fp16=torch.cuda.is_available(),
    logging_steps=100,

    report_to="none"
)

In [8]:
test_prompts = [
    "Все мысли, которые имеют огромные последствия",
    "Сила войска зависит от его духа",
    "Мысль о том, что он принес страдания",
    "Человек сознает себя свободным",
    "Что бы ни случилось, я всегда буду",
    "Любовь мешает смерти",
    "Нет, жизнь не кончена",
    "Всякая мысль, даже самая простая",
    "Война не любезность, а самое гадкое дело",
    "Чтобы жить честно"
]

In [9]:
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl
import torch

class GenerationCallback(TrainerCallback):
    """
    Custom callback to evaluate model generation quality during training.
    It generates text based on a list of test prompts at the end of each evaluation phase.
    """
    def __init__(self, test_prompts, tokenizer, model):
        self.test_prompts = test_prompts
        self.tokenizer = tokenizer
        self.model = model

    def on_evaluate(self, args, state, control, **kwargs):

        # Switch model to evaluation mode (disables dropout, etc.)
        self.model.eval()
        device = self.model.device

        print(f"\n--- Generating samples at step {state.global_step} ---")

        for prompt in self.test_prompts:
            # Tokenize the prompt and move to the active device
            inputs = self.tokenizer(prompt, return_tensors="pt").input_ids.to(device)

            with torch.no_grad():
                output = self.model.generate(
                    inputs,
                    max_length=inputs.shape[1] + 150, # Generate up to 150 *new* tokens
                    do_sample=True,    # Enable sampling strategy (probabilistic generation)
                    temperature=0.7,   # Control randomness: lower = more deterministic, higher = more creative
                    top_k=50,          # Top-K sampling: limit selection to the top 50 probability tokens
                )

            # Decode the generated tokens back to text
            generated_text = self.tokenizer.decode(output[0], skip_special_tokens=True)

            print(f"Prompt: \"{prompt}\"")
            print(f"Generated: {generated_text}\n")

        # Switch model back to training mode to resume training correctly
        self.model.train()

In [10]:
generation_callback = GenerationCallback(test_prompts, tokenizer, model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    callbacks=[generation_callback]
)

trainer.train()

/tmp/ipython-input-1008094080.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 0, 'pad_token_id': 2}.


Step,Training Loss,Validation Loss
100,7.562600,6.886019
200,6.597500,6.351955
300,6.231700,6.097020
400,6.004000,5.932364
500,5.872200,5.819438
600,5.767800,5.741858
700,5.691600,5.691276
800,5.650300,5.663435


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



--- Generating samples at step 100 ---
Prompt: "Все мысли, которые имеют огромные последствия"
Generated: Все мысли, которые имеют огромные последствияПс,

Prompt: "Сила войска зависит от его духа"
Generated: Сила войска зависит от его духаВим; и, что на-то!

Prompt: "Мысль о том, что он принес страдания"
Generated: Мысль о том, что он принес страдания-- Да, что я не она, и что я в она, и не ты.

Prompt: "Человек сознает себя свободным"
Generated: Человек сознает себя свободным- Ну, и как на в ее и, я в этом с

Prompt: "Что бы ни случилось, я всегда буду"
Generated: Что бы ни случилось, я всегда будуВ-нибудь.

Prompt: "Любовь мешает смерти"
Generated: Любовь мешает смертиВ в поать в, а вы, что и, и, у,

Prompt: "Нет, жизнь не кончена"
Generated: Нет, жизнь не кончена- К-то, что, не вы на меня.

Prompt: "Всякая мысль, даже самая простая"
Generated: Всякая мысль, даже самая простая- К не мне, и в ее, но.

Prompt: "Война не любезность, а самое гадкое дело"
Generated: Война не любезность,

TrainOutput(global_step=846, training_loss=6.142734753042812, metrics={'train_runtime': 5508.2999, 'train_samples_per_second': 9.801, 'train_steps_per_second': 0.154, 'total_flos': 2.512029601706803e+16, 'train_loss': 6.142734753042812, 'epoch': 3.0})

Post-train SFT

In [11]:
from datasets import load_dataset, Dataset
import os

raw_dataset = load_dataset("d0rj/alpaca-cleaned-ru", split="train")



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/760 [00:00<?, ?B/s]

data/train-00000-of-00001-c503683bee003a(…):   0%|          | 0.00/36.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [12]:
print(raw_dataset[0])

{'input': '', 'instruction': 'Дайте три совета, как оставаться здоровым.', 'output': '1. Соблюдайте сбалансированную и питательную диету. Убедитесь, что в ваш рацион входят разнообразные фрукты и овощи, нежирный белок, цельнозерновые продукты и полезные жиры. Это помогает обеспечить ваш организм необходимыми питательными веществами для оптимального функционирования и может помочь предотвратить хронические заболевания.\n\n2. Занимайтесь регулярной физической активностью. Упражнения имеют решающее значение для поддержания крепких костей, мышц и здоровья сердечно-сосудистой системы. Старайтесь уделять не менее 150 минут умеренным аэробным упражнениям или 75 минут интенсивным упражнениям каждую неделю.\n\n3. Высыпайтесь. Достаточное количество качественного сна имеет решающее значение для физического и психического благополучия. Он помогает регулировать настроение, улучшать когнитивные функции и поддерживает здоровый рост и иммунную функцию. Старайтесь спать 7-9 часов каждую ночь.'}


In [13]:
import os

def format_alpaca_to_dialogue(example):
    """
    Transforms an Alpaca-style example (instruction, input, output) into a single text string.
    """
    system = ""
    if example["input"]:
        system = example['input'] + "\n"

    user = example['instruction']
    assistant = example['output']

    # Combine fields into a single sequence
    full_text = system + user + "\n\n" + assistant

    return {"text": full_text}

# Apply formatting to the entire dataset
formatted_dataset = raw_dataset.map(
    format_alpaca_to_dialogue,
    num_proc=os.cpu_count(),
    # Drop original columns (instruction, input, output) to keep only the processed 'text'
    remove_columns=raw_dataset.column_names
)

print(f"Total number of examples: {len(formatted_dataset)}")
print("Sample of the first formatted dialogue:")
print(formatted_dataset[0]["text"])

split_datasets = formatted_dataset.train_test_split(test_size=0.05, seed=42)

train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]

print(f"\nDataset Split: Train={len(train_dataset)}, Validation={len(eval_dataset)}")

Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

Total number of examples: 51760
Sample of the first formatted dialogue:
Дайте три совета, как оставаться здоровым.

1. Соблюдайте сбалансированную и питательную диету. Убедитесь, что в ваш рацион входят разнообразные фрукты и овощи, нежирный белок, цельнозерновые продукты и полезные жиры. Это помогает обеспечить ваш организм необходимыми питательными веществами для оптимального функционирования и может помочь предотвратить хронические заболевания.

2. Занимайтесь регулярной физической активностью. Упражнения имеют решающее значение для поддержания крепких костей, мышц и здоровья сердечно-сосудистой системы. Старайтесь уделять не менее 150 минут умеренным аэробным упражнениям или 75 минут интенсивным упражнениям каждую неделю.

3. Высыпайтесь. Достаточное количество качественного сна имеет решающее значение для физического и психического благополучия. Он помогает регулировать настроение, улучшать когнитивные функции и поддерживает здоровый рост и иммунную функцию. Старайтесь спать 7-9 ч

In [14]:
print(formatted_dataset[0])

{'text': 'Дайте три совета, как оставаться здоровым.\n\n1. Соблюдайте сбалансированную и питательную диету. Убедитесь, что в ваш рацион входят разнообразные фрукты и овощи, нежирный белок, цельнозерновые продукты и полезные жиры. Это помогает обеспечить ваш организм необходимыми питательными веществами для оптимального функционирования и может помочь предотвратить хронические заболевания.\n\n2. Занимайтесь регулярной физической активностью. Упражнения имеют решающее значение для поддержания крепких костей, мышц и здоровья сердечно-сосудистой системы. Старайтесь уделять не менее 150 минут умеренным аэробным упражнениям или 75 минут интенсивным упражнениям каждую неделю.\n\n3. Высыпайтесь. Достаточное количество качественного сна имеет решающее значение для физического и психического благополучия. Он помогает регулировать настроение, улучшать когнитивные функции и поддерживает здоровый рост и иммунную функцию. Старайтесь спать 7-9 часов каждую ночь.'}


In [ ]:
pip install trl

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
import torch

MODEL_NAME = "Qwen/Qwen2.5-0.5B"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    # Optimize memory usage with BF16 or FP16 precision
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    # Automatically map model layers to available devices (GPU/CPU sharding)
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer.pad_token = tokenizer.eos_token # Set PAD token to EOS token (Crucial for SFT to ensure correct masking and stop conditions)
tokenizer.padding_side = "right"

In [17]:
from transformers import TrainingArguments, Trainer, PreTrainedTokenizerFast
from trl import SFTTrainer, SFTConfig

CONTEXT_LENGTH = 1024
BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = 16


training_args = TrainingArguments(

    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    weight_decay=0.01,

    num_train_epochs=3,
    learning_rate=2e-5,

    eval_strategy="steps", # Estimation on a validation set
    per_device_eval_batch_size=BATCH_SIZE,

    #fp16=torch.cuda.is_available(),
    fp16=False, # <-- switch off fp16
    bf16=True,
    logging_steps=100,

    report_to="none",
)


In [18]:
questions_rus = [
    "сколько планет в нашей солнечной системе?",
    "расскажи стих",
    "когда собирать крыжовник?",
    "Как быстро выучить новый язык?"
  ]

In [19]:
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl
import torch

class GenerationCallback(TrainerCallback):
    """
    Custom callback to visually evaluate model performance during training.
    It triggers text generation on specific test prompts at the end of each evaluation phase.
    """
    def __init__(self, test_prompts, tokenizer, model):
        self.test_prompts = test_prompts
        self.tokenizer = tokenizer
        self.model = model

    def on_evaluate(self, args, state, control, **kwargs):

        self.model.eval() # Switch model to evaluation mode (disables dropout, updates batch norm stats)
        device = self.model.device

        print(f"\n Generating samples at step {state.global_step}")

        for prompt in self.test_prompts:
            # Tokenize the prompt and ensure tensors are on the correct device (GPU/CPU)
            inputs = self.tokenizer(prompt, return_tensors="pt").input_ids.to(device)

            with torch.no_grad():
                output = self.model.generate(
                    inputs,
                    max_length=inputs.shape[1] + 350, # Generate up to 350 new tokens
                    do_sample=True,    # Enable sampling strategy (probabilistic generation)
                    temperature=0.7,   # Control randomness: 0.7 balances creativity and coherence
                    top_k=50,          # Top-K sampling: restricts candidate tokens to the 50 highest probability options
                )

            # Decode the generated output
            generated_text = self.tokenizer.decode(output[0], skip_special_tokens=True)

            print(f"Prompt: \"{prompt}\"")
            print(f"Response: {generated_text}\n")

        # back to training mode
        self.model.train()

In [ ]:
generation_callback = GenerationCallback(questions_rus, tokenizer, model)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[generation_callback],
)

trainer.train()

Adding EOS to train dataset:   0%|          | 0/49172 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/49172 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/49172 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/2588 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/2588 [00:00<?, ? examples/s]

...

Prompt: "сколько планет в нашей солнечной системе?"
Response: сколько планет в нашей солнечной системе? Вместо этого напишите описание и объясните, почему это неправильно.

Планеты в нашем солнечном системе — это осязаемые, светопрозрачные и гравитирующие объекты в системе, состоящей из двух планет и нескольких галактик. Несколько планет в нашей системе известны как звездные планеты, а более крупные — планеты, которая были бы обитающими на Земле. По мере увеличения размера планеты она также прокрашивает более широкий диапазон галактик, что означает, что она может обитать на разных галактических сегментах. Когда планета обретает более широкий диапазон галактик, она может стать более густой и более вспышкой света, что приводит к более длинному диапазону света и более низким температурам. Это может привести к более морозным и морякам условиям, чем когда-либо, и может привести к более низким температурам планеты. Однако не менее важным фактором является то, что планеты имеют различные сост

Both `max_new_tokens` (=2048) and `max_length`(=360) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Prompt: "расскажи стих"
Response: расскажи стихотворению о двух мечтах

Мечта — это гордость, когда ты чувствуешь себя в своем праве,
Мечта — это желание, когда ты чувствуешь себя в себе.
Мечта — это навык, когда ты ведешь себя в соответствии со своей волей,
В ней нет ничего, чего можно предпринять с другой стороны.

Мечта — это энтузиазм, когда ты начинаешь с небольшим и начинаешь изучать каждый день,
Мечта — это путь, когда ты начинаешь с небольшим и начинаешь идти дальше.

Мечта — это чувство, когда ты чувствуешь себя в своем праве и в своем праве,
Когда ты знаешь, что ты можешь делать, когда тебе это нужно.



Both `max_new_tokens` (=2048) and `max_length`(=358) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Prompt: "когда собирать крыжовник?"
Response: когда собирать крыжовник?.
Расскажите, чем может быть связанное с этим предложение.

Крыжовник — это небольшая ветвь деревья, которая прикреплена к стволу ее родной деревца. Для собираи крыжовника обычно используется деревянная махина, а также крепление рукоятка. Махина должна быть надежной, толстой и сильной, позволяя крепко придерживать крыжовник. Крыжовника можно собирать в качестве доски для ремонта и ухода за ведро, или для быстрого и эффективного сбора.

Prompt: "Как быстро выучить новый язык?"
Response: Как быстро выучить новый язык? Какие уроки нужны?

Вот шаги, которые могут помочь вам быстро и эффективно учиться на новый языке:

1. Соберите необходимые ресурсы. Обучение на новый язык требует знаний о языке в целом, его лингвической структуре, навыках и личных предпочтениях. Соберите ресурсы, такие как книги, учебники, онлайн-курсы, интерактивные программы для интервью, видеоклассиконы и другие.

2. Определите цель и визуализируйте

In [ ]:
# Мне не хватило вычислительных мощностей, чтобы провести запланированных 3 эпохи, а разные способы "облегчения" работы с gpu после нескольких попыток внедрить не получилось :(
# Но даже после сотого шага видно, что ответы относительно осмысленные, по крайней мере на русском языке!